# quake 3


## file management



### managing zips/pk3s



#### find pk3 files in zips?


In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')

function readZip(zipFile, cb) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        // Take a look at the files
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        var pk3s = Object.values(zip.entries())
            .filter(entry => entry.name.includes('.pk3'))
        
        return importer.runAllPromises(pk3s.map((pk3, i) => new Promise((resolve) => {
            var pk3Path = '/Applications/ioquake3/bestmaps/' + path.basename(pk3.name);
            if(!fs.existsSync(pk3Path)) {
                zip.extract(pk3.name, pk3Path, err => {
                    if(err) {
                        console.log('Extract error ' + err)
                        return resolve()
                    }
                    console.log('Extracted ' + pk3.name);
                    resolve(pk3Path)
                });
            }
            resolve()
        }))).then(() => {
            zip.close()
            cb()
        })
    });
    
    zip.on('error', err => { cb() });
}

function extractAll(root) {
    var results = listInProject(root || '/Users/briancullinan/planet_quake_resources', '**/*.zip')
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readZip(r, resolve)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}

module.exports = extractAll;


#### find only bsps in map paks?



In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')


function readPak(zipFile, cb) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        for (const entry of Object.values(zip.entries())) {
            if(entry.name.includes('.bsp')) {
                console.log(entry.name)
                /*var pk3Path = '/Applications/ioquake3/bestmaps/' + path.basename(pk3.name);
                if(!fs.existsSync(pk3Path)) {
                    zip.extract(pk3.name, pk3Path, err => {
                        console.log((err ? 'Extract error ' : 'Extracted ') + pk3.name);
                        zip.close();
                        cb(zipFile);
                    });
                    return;
                }*/
            }
            if(entry.name.includes('levelshots/') && entry.name !== 'levelshots/') {
                var levelPath = '/Applications/ioquake3/bestmaps/levelshots/' + path.basename(entry.name)
                if(!fs.existsSync(levelPath)) {
                    zip.extract(entry.name, levelPath, err => {
                        console.log((err ? 'Extract error ' : 'Extracted ') + entry.name);
                        zip.close();
                        cb(zipFile);
                    });
                    return;
                }
            }
        }
        zip.close()
        cb();
    });
    
    zip.on('error', err => { cb() });
}


function extractPaks() {
    var results = listInProject('/Applications/ioquake3/bestmaps', '**/*.pk3')
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readPak(r, resolve)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}


module.exports = extractPaks;


#### list bsps in a pak?

For that annoying problem when you download a map pk3 and it's a different name than the map.


In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');

function listBsps(zipFile) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        for (const entry of Object.values(zip.entries())) {
            if(entry.name.includes('.bsp')) {
                console.log(entry.name)
            }
        }
    });
    
    zip.on('error', err => {  });
}

module.exports = listBsps;


#### unpack entire folder of pk3s?

unpack pk3s?


In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')
var mkdirpSync = importer.import('mkdirp')

function readPak(zipFile, cb, outdir) {
    if(!outdir) {
        outdir = '/Users/briancullinan/.q3a/unpacked/' + path.basename(zipFile) + 'dir'
    }
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        var index = Object.values(zip.entries())
        return Promise.all(index.map(entry => {
            var levelPath = path.join(outdir, entry.name)
            mkdirpSync(path.dirname(levelPath))
            return new Promise(resolve => {
                zip.extract(entry.name, levelPath, err => {
                    console.log((err ? 'Extract error ' : 'Extracted ') + entry.name);
                    resolve(zipFile);
                });
            })
        })).then(() => {
            zip.close();
            cb()
        })
    });
    
    zip.on('error', err => { cb() });
}


function extractPaks(inpaks, outdir) {
    var results = listInProject(path.dirname(inpaks), '**/' + path.basename(inpaks))
    results.sort()
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readPak(r, resolve, outdir)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}


module.exports = extractPaks;


### finding models and textures



### converting graphics and audio

Pretty tired of messing around with bash conversions. Call imagemagick commands from Node because strings and file names are easier to manipulate in JS.


#### convert images that don't have alpha

convert nonalpha?


In [ ]:
var path = require('path')
var fs = require('fs')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var execCmd = importer.import('spawn child process')
var mkdirpSync = importer.import('mkdirp')
var { chext, chroot } = importer.import('changing file names')
var {imageTypes} = importer.import('quake 3 file whitelist')

async function convertNonAlpha(root) {
    var result = []
    var output = path.join(path.dirname(root), path.basename(root) + '-converted')
    var files = await glob(imageTypes.map(a => '**/*' + a), root)
    for(var i = 0; i < files.length; i++) {
        var outFile, inFile = files[i]
        var alphaCmd = await execCmd(`identify -format '%[opaque]' '${inFile}'`, {quiet: true})
        // if it is alpha
        if(alphaCmd[0].localeCompare('False') === 0) {
            // convert everything else to png to support transparency
            outFile = chroot(chext(inFile, '.png'), root, output)
        } else {
            // if a jpg already exists, use that file for convert
            if(fs.existsSync(chext(inFile, '.jpg'))) {
                inFile = chext(inFile, '.jpg')
            }
            // transfer low quality jpeg instead
            outFile = chroot(chext(inFile, '.jpg'), root, output)
        }
        mkdirpSync(path.dirname(outFile))
        // convert, baseq3 already includes jpg
        await execCmd(`convert -strip -interlace Plane -sampling-factor 4:2:0 -quality 50% "${inFile}" "${outFile}"`, {quiet: true})
        result.push(outFile)
    }
    return result
}


module.exports = convertNonAlpha

#### convert quake 3 audio?



In [ ]:
var path = require('path')
var fs = require('fs')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var execCmd = importer.import('spawn child process')
var mkdirpSync = importer.import('mkdirp')
var { chext, chroot } = importer.import('changing file names')
var {audioTypes} = importer.import('quake 3 file whitelist')

async function convertAudio(root) {
    var result = []
    var output = path.join(path.dirname(root), path.basename(root) + '-converted')
    
    var files = await glob(audioTypes.map(a => '**/*' + a), root)
    for(var i = 0; i < files.length; i++) {
        var outFile, inFile = files[i]
        outFile = chroot(chext(inFile, '.opus'), root, output)
        mkdirpSync(path.dirname(outFile))
        await execCmd(`opusenc --bitrate 24 --vbr "${inFile}" "${outFile}"`, {quiet: false})
        result.push(outFile)
    }
    return result
}

module.exports = convertAudio


#### copy quake 3 script files?


In [ ]:
var path = require('path')
var fs = require('fs')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var mkdirpSync = importer.import('mkdirp')
var { chext, chroot } = importer.import('changing file names')
var {fileTypes} = importer.import('quake 3 file whitelist')

async function convertScripts(root) {
    var result = []
    var output = path.join(path.dirname(root), path.basename(root) + '-converted')
    var files = await glob(fileTypes.map(a => '**/*' + a), root)
    for(var i = 0; i < files.length; i++) {
        var inFile = files[i]
        var outFile = chroot(inFile, root, output)
        if(!fs.existsSync(outFile)) {
            mkdirpSync(path.dirname(outFile))
            await fs.copyFileSync(inFile, outFile)
            result.push(outFile)
        }
    }
    return result
}

module.exports = convertScripts


#### whitelist for all quake 3 file types

quake 3 file whitelist?


TODO: scan lots of mods and file files that aren't in the list

TODO: scan the quake 3 code and look for extensions, particularly Unvanquished which adds lots of file types for compatibility.


In [ ]:

var imageTypes = [
    '.png',
    '.jpg',
    '.jpeg',
    '.tga',
    '.gif',
    '.pcx',
    '.webp',
]

var audioTypes = [
    '.opus',
    '.wav',
    '.mp3',
]

var sourceFiles = [
    '.map',
    '.scc',
]

var fileTypes = [
    '.cfg',
    '.qvm',
    '.bot',
    '.txt',
    '.bsp',
    '.aas',
    '.md3',
    '.md5',
    '.shader',
    '.skin',
    '.pk3',
    '.c',
    '.h',
    '.config',
    '.menu'
]


var knownDirs = [
    'scripts/',
    'botfiles/',
	'fonts/',
	'gfx/',
    'hud/',
	'icons/',
	'include/',
	'menu/',
	'models/',
	'music/',
	'powerups/',  // powerup shaders
	'sprites/',
	'sound/',
	'ui/',
    'maps/',
    'textures/',
    
]

module.exports = {
    imageTypes,
    audioTypes,
    sourceFiles,
    fileTypes,
    knownDirs,
}



#### test pack creation with results from ]\imagelist

quake 3 menu image list?


In [ ]:
var fs = require('fs')
var importer = require('../Core')
var { chext, chroot } = importer.import('changing file names')
var {glob} = importer.import('glob files')
var mkdirpSync = importer.import('mkdirp')

var imagelist = `
  68:   64x  64 RGB       18kb gfx/misc/flare.tga
  69:   64x  64 RGB       18kb gfx/misc/sun.tga
  70:  256x 256 RGBA     256kb gfx/2d/bigchars.tga
  71:  128x 128 RGBA      64kb gfx/misc/console01.tga
  72:  128x 128 RGB       48kb gfx/misc/console02.tga
  73:  256x 256 RGBA     256kb menu/art/font1_prop.tga
  74:  256x 256 RGBA     256kb menu/art/font1_prop_glo.tga
  75:  256x 256 RGBA     256kb menu/art/font2_prop.tga
  76:   32x  32 RGBA       4kb menu/art/3_cursor2
  77:   16x  16 RGBA    1024b  menu/art/switch_on
  78:   16x  16 RGBA    1024b  menu/art/switch_off
  79:  512x 512 RGB      768kb textures/sfx/logo512.tga
  80:    8x   8 RGB      192b  gfx/colors/black.tga
  81:  128x  16 RGBA       8kb menu/art/slider2
  82:   16x  32 RGBA       2kb menu/art/sliderbutt_0
  83:   16x  32 RGBA       2kb menu/art/sliderbutt_1
  84:  256x 128 RGBA     128kb models/mapobjects/banner/q3banner02.tga
  85:   64x 128 RGBA      32kb models/mapobjects/banner/q3banner02x.tga
  86:  128x 128 RGB       48kb textures/sfx/firegorre2.tga
  87:  128x 128 RGB       48kb textures/sfx/bolts.tga
  88:  256x 256 RGBA     256kb menu/art/maps_select
  89:  128x 129 RGBA      64kb menu/art/maps_selected
  90:   16x 128 RGBA       8kb menu/art/narrow_0
  91:   16x 128 RGBA       8kb menu/art/narrow_1
  92:   64x  64 RGB       12kb menu/art/unknownmap
  93:  128x 128 RGBA      64kb menu/art/level_complete1
  94:  128x 128 RGBA      64kb menu/art/level_complete2
  95:  128x 128 RGBA      64kb menu/art/level_complete3
  96:  128x 128 RGBA      64kb menu/art/level_complete4
  97:  128x 128 RGBA      64kb menu/art/level_complete5
  98:  128x  64 RGBA      32kb menu/art/back_0
  99:  128x  64 RGBA      32kb menu/art/back_1
 100:  128x  64 RGBA      32kb menu/art/fight_0
 101:  128x  64 RGBA      32kb menu/art/fight_1
 102:  128x  64 RGBA      32kb menu/art/reset_0
 103:  128x  64 RGBA      32kb menu/art/reset_1
 104:  128x  64 RGBA      32kb menu/art/skirmish_0
 105:  128x  64 RGBA      32kb menu/art/skirmish_1
 106:   64x  64 RGBA      16kb menu/medals/medal_accuracy
 107:   64x  64 RGBA      16kb menu/medals/medal_impressive
 108:   64x  64 RGBA      16kb menu/medals/medal_excellent
 109:   64x  64 RGBA      16kb menu/medals/medal_gauntlet
 110:   64x  64 RGBA      16kb menu/medals/medal_frags
 111:   64x  64 RGBA      16kb menu/medals/medal_victory
 112:   64x  64 RGBA      16kb models/players/sarge/icon_default.tga
 113:  143x 128 RGB       53kb levelshots/q3dm0.tga
 114:   64x  64 RGBA      16kb models/players/crash/icon_default.tga
`

var shaderlist = `
1     gen : gfx/2d/sunflare
1   E gen : gfx/2d/bigchars
1   E gen : white
2   E gen : console
1     gen : menu/art/font1_prop
1     gen : menu/art/font1_prop_glo
1     gen : menu/art/font2_prop
1     gen : menu/art/3_cursor2
1     gen : menu/art/switch_on
1     gen : menu/art/switch_off
1   E gen : menuback
1   E gen : menubacknologo
1     gen : menu/art/slider2
1     gen : menu/art/sliderbutt_0
1     gen : menu/art/sliderbutt_1
3   E gen : models/mapobjects/banner/q3banner02
3   E gen : models/mapobjects/banner/q3banner04
1     gen : menu/art/maps_select
1     gen : menu/art/maps_selected
1     gen : menu/art/narrow_0
1     gen : menu/art/narrow_1
1     gen : menu/art/unknownmap
1     gen : menu/art/level_complete1
1     gen : menu/art/level_complete2
1     gen : menu/art/level_complete3
1     gen : menu/art/level_complete4
1     gen : menu/art/level_complete5
1     gen : menu/art/back_0
1     gen : menu/art/back_1
1     gen : menu/art/fight_0
1     gen : menu/art/fight_1
1     gen : menu/art/reset_0
1     gen : menu/art/reset_1
1     gen : menu/art/skirmish_0
1     gen : menu/art/skirmish_1
1     gen : menu/medals/medal_accuracy
1     gen : menu/medals/medal_impressive
1     gen : menu/medals/medal_excellent
1     gen : menu/medals/medal_gauntlet
1     gen : menu/medals/medal_frags
1     gen : menu/medals/medal_victory
1     gen : models/players/sarge/icon_default
1     gen : levelshots/q3dm0
1     gen : models/players/crash/icon_default
`

async function getImages(root) {
    var result = []
    var output = path.join(path.dirname(root), path.basename(root) + '-menu')
    var images = imagelist
        .split('\n').map(l => l.split(/[\t\s]+/ig)[6])
        .filter(i => i)
        .map(i => '**/' + chext(i, '') + '*')
    var files = await glob(images, root)
    for(var i = 0; i < files.length; i++) {
        var outFile = chroot(files[i], root, output)
        mkdirpSync(path.dirname(outFile))
        fs.copyFileSync(files[i], outFile)
        result.push(outFile)
    }
    
    
    return result
}

module.exports = getImages


### TODO: organize textures, find overrides (accidental and intentional)

Does TrenchBroom really require everything be in 1 folder?


TODO: all world, object and player textures run through waifu2x and SFTGAN


## convert quake 2 map to quake 3


#### the code

convert quake 2 map to quake 3?


In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')
var replaceClasses = importer.import('replace known classes in map')
var replaceEntities = importer.import('replace entities in map')
var replaceTextures = importer.import('replace common textures in map')
var scaleMap = importer.import('scale quake map')
var {addSkybox} = importer.import('add skybox to map')
var {MAPS_HINTS, addHints} = importer.import('draw hints in map')

function convertMap(fileName) {
    var file;
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
    }
    
    file = replaceEntities(file)
    
    file = replaceClasses(file)
    
    // replace water with foggy water
    
    // replace message with target print
    
    // replace duplicate textures / don't interfere with basepack
    file = replaceTextures(file)
    
    // replace barrels with rocket launchers
    
    // replace animations and generate proper shaders
    // scale map 1.2x
    file = scaleMap(file, 1.15)
    
    //file = addSkybox(file)
    
    // scale lighting with q3map2 bsp options
    var newFile = fileName.substr(0, fileName.length - path.extname(fileName).length) + '.map'
    console.log(`writing ${newFile}`)
    fs.writeFileSync(newFile, file)
}

module.exports = convertMap;


#### scale quake map?



In [ ]:
var fs = require('fs')
var DIGITS = 100.0

function scaleMap(file, amount) {
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S]*?\}/ig, file)
    
    // replace all brushes with scaled values
    brushes.forEach(b => {
        var newBrush = b
        newBrush = newBrush.replace(/\(((\s*[0-9\.-]+\s*)*)\)/ig, (str, $1) => {
            return '( ' + $1.trim().split(/\s+/ig)
                .map(n => Math.round((n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
                .join(' ') + ' )'
        })
        // scale the texture on the brush
        newBrush = newBrush.replace(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+\s*){5,8})/ig, (str, $1, $2) => {
            return ') ' + $1 + ' ' + $2.trim().split(/\s+/ig)
                .map((n, i) => i >= 5 || i <= 2 ? n : (Math.round((n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim())) * amount * DIGITS) / DIGITS))
                .join(' ') + '\n'
        })
        file = file.replace(b, newBrush)
    })
    
    // replace all origins with scaled
    // TODO: make this a function
    var origins = importer.regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
    origins.forEach($1 => {
        var newOrigin = $1.trim().split(/\s+/ig)
            .map(n => Math.round((n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
            .join(' ')
        file = file.replace(new RegExp('"origin"\\s+"' + $1 + '"', 'ig'), '"origin" "' + newOrigin + '"')
    })
    var lips = importer.regexToArray(/"lip"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
    lips.forEach($1 => {
        var newOrigin = $1.trim().split(/\s+/ig)
            .map(n => Math.round((n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
            .join(' ')
        file = file.replace(new RegExp('"lip"\\s+"' + $1 + '"', 'ig'), '"lip" "' + newOrigin + '"')
    })
    
    // scale models
    var models = importer.regexToArray(/\{[\s\S^\}\{]*?"classname"\s+"misc_model"[\s\S^\}\{]*?\}/ig, file)
    models.forEach(m => {
        var newModel = m
        if(m.match(/"modelscale"/i)) {
            var match = (/"modelscale"\s+"([^"]*?)"/ig).exec(m)
            var scaled = (match[1].includes('.')
                ? parseFloat(match[1])
                : parseInt(match[1])) * amount
            newModel = newModel.replace(match[0], '"modelscale" "' + scaled + '"')
        } else {
            newModel = newModel.replace(/"classname"\s+"misc_model"/ig,
                                        '"classname" "misc_model"\n "modelscale" "' + amount + '"')
        }
        file = file.replace(m, newModel)
    })
    
    return file
}

module.exports = scaleMap


#### translate quake map?



In [ ]:
var importer = require('../Core')


function translateMap(file, amount) {
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[^\{\}]*?\}/ig, file)
    
    // replace all brushes with scaled values
    brushes.forEach(b => {
        var newBrush = b
        newBrush = newBrush.replace(/\(((\s*[0-9\.-]+\s*)*)\)/ig, (str, $1) => {
            return '( ' + $1.trim().split(/\s+/ig)
                .map((n, i) => (n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim())) + amount[i])
                .join(' ') + ' )'
        })
        file = file.replace(b, newBrush)
    })
    
    // replace all origins with scaled
    // TODO: make this a function
    var origins = importer.regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
    origins.forEach($1 => {
        var newOrigin = $1.trim().split(/\s+/ig)
            .map((n, i) => (n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim())) + amount[i])
            .join(' ')
        file = file.replace(new RegExp('"origin"\\s+"' + $1 + '"', 'ig'), '"origin" "' + newOrigin + '"')
    })
    
    return file
}

module.exports = translateMap


#### replace common textures in map?


In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')

var common = [
  'areaportal',    'botclip',
  'caulk',         'clip',
  'clusterportal', 'cushion',
  'donotenter',    'full_clip',
  'hint',          'ladderclip',
  'ladderclip',    'lightgrid',
  'metalclip',     'missileclip',
  'nodraw',        'nodrawnonsolid',
  'nodrop',        'nolightmap',
  'origin',        'qer_mirror',
  'qer_portal',    'slick',
  'terrain',       'trigger',
  'weapclip',      'white'
];
var textures = {
    
}
function replaceTextures(file) {
    var used = []
    
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // get a list of common textures
    if(!common) {
        var files = fs.readdirSync('/Users/briancullinan/planet_quake_data/quake3-baseq3/common-spog.pk3dir/textures/common')
        common = files.map(f => f.substr(0, f.length - path.extname(f).length));
    }
    
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S]*?\}/ig, file)
    
    // replace all brushes with textures from common or berserker overrides
    brushes.forEach(b => {
        var newBrush = b
        Object.keys(textures).forEach(k => {
            newBrush = newBrush.replace(new RegExp('\\) ([^\\/\\)\\(]*?\\/)*?' + k, 'ig'),
                                        (str, $1) => (') ' + textures[k]))
        })
        common.forEach(k => {
            newBrush = newBrush.replace(new RegExp('\\) ([^\\/\\)\\(]*?\\/)*?' + k, 'ig'),
                                (str, $1) => (') common/' + (k === 'clip' ? 'hint' : k)))
        })
        // fix water, don't mix textures because
        //   trenchbroom rearranges verteces so water isn't always applied
        if(newBrush.includes('wter')) {
            newBrush = newBrush
                .replace(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+){5,8})/igm,
                         (str, $1, $2) => (') e1u1/bluwter ' + $2))
        }
        var texts = importer.regexToArray(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+){5,8})/igm, b, 1)
        texts.forEach($1 => {
            if(used.indexOf(path.dirname($1)) === -1) used.push(path.dirname($1))
        })
        
        file = file.replace(b, newBrush)
    })
    
    used = used.map(u => 'textures/' + u.replace('^\/|\/$|^\s*|\s*$', ''))
    file = file.replace(/"classname" "worldspawn"/ig,`"classname" "worldspawn"
"_tb_textures" "textures/common;${used.join(';')}"`)

    return file
}

module.exports = replaceTextures


#### replace known classes in map?



In [ ]:
var fs = require('fs')

var classes = {
    weapon_chaingun:                'weapon_lightning',
    misc_teleporter:                'trigger_teleport',
    weapon_hyperblasters:           'weapon_plasmagun',
    weapon_supershotgun:            'weapon_shotgun',
    info_teleport_destination:      'target_position',
    item_armor1:                    'item_armor_combat',
    item_armor2:                    'item_armor_combat',
    item_armorInv:                  'item_armor_body',
    item_artifact_envirosuit:       'item_enviro',
    item_artifact_invisibility:     'item_invis',
    item_artifact_super_damage:     'item_quad',
    item_cells:                     'ammo_lightning',
    item_rockets:                   'ammo_rockets',
    item_shells:                    'ammo_shells',
    item_spikes:                    'ammo_cells',
    weapon_nailgun:                 'weapon_plasmagun',
    weapon_supernailgun:            'weapon_plasmagun',
    weapon_supershotgun:            'weapon_shotgun',
    // replace target_help with target_print
    target_help:                    'target_print',
    func_door_rotating:             'func_rotating'
}
function replaceClasses(file) {
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // replace explosions with misc_model
    file = file.replace(/"classname" "target_explosion"/ig,
                        '"classname" "misc_model"\n "model" "models/objects/r_explode/tris.md2"')
    // TODO: replace explosions with animated delay
    // replace tigger_once with multiple wait -1
    file = file.replace(/"classname" "trigger_once"/ig,
                        '"classname" "trigger_multiple"\n "wait" "-1"')
    // replace classes
    Object.keys(classes).forEach(k => {
        file = file.replace(new RegExp('"classname" "' + k + '"', 'ig'),
                            '"classname" "' + classes[k] + '"')
    })
    return file
}

module.exports = replaceClasses


#### replace entities in map?



In [ ]:
var fs = require('fs')
var importer = require('../Core')

var MATCH_MODELS = /QUAKED\s+([^\s]*)\s+[\s\S]*?model="([^"]*)"/i;

var ents;
function replaceEntities(file, entities) {
    if(!entities) {
        entities = '/Users/briancullinan/planet_quake_data/quake2-baseq2/scripts/entities.def'
    }
    
    // parse entities definition
    if(!ents) {
        var entityStr = fs.readFileSync(entities).toString('utf-8')
        var matches = importer.regexToArray(/QUAKED\s+.*/ig, entityStr)
        ents = matches.reduce((obj, m, i, arr) => {
            var start = entityStr.indexOf(m)
            var end = entityStr.indexOf(arr[i + 1])
            if(end === -1) end = entityStr.length
            var groups = MATCH_MODELS.exec(entityStr.substring(start, end))
            if(!groups) return obj
            obj[groups[1]] = groups[2]
            return obj
        }, {})
    }
    
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // replace entities
    file = file.replace(/"misc_([^"]*)"/ig, ($0) => `"misc_model"\n "model" ${$0}`)
    
    var noises = importer.regexToArray(/noise.*?"\s*"(.*?)"/ig, file, false)
    noises.forEach(n => {
        if(!n[1].includes('sound/')) {
            file = file.replace(n[0], `noise" "sound/${n[1]}"`)
        }
    })
    
    // replace misc_ with models from ent
    Object.keys(ents).forEach(k => {
        if(!k.includes('misc_')) return
        file = file.replace(new RegExp('"' + k + '"', 'ig'), '"' + ents[k] + '"')
    })
    return file
}

module.exports = replaceEntities;


#### add a skybox and remove sky entities?

add skybox to map?

Because quake 2 maps leak somewhere during this process.


In [ ]:
var importer = require('../Core')
var {doIntersect} = importer.import('brush to vertex')

function getBounds(file) {
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S]*?\}/ig, file)
   
    brushes = brushes.map(b => {
        var points = importer
            .regexToArray(/\(((\s*[0-9\.-]+\s*)*)\)/ig, b, 1)
            .map(m => m.trim().split(/\s+/ig)
                .map(n => (n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim()))))
        
        return [[
            Math.min.apply(null, points.map(b => b[0])),
            Math.min.apply(null, points.map(b => b[1])),
            Math.min.apply(null, points.map(b => b[2]))
        ], [
            Math.max.apply(null, points.map(b => b[0])),
            Math.max.apply(null, points.map(b => b[1])),
            Math.max.apply(null, points.map(b => b[2]))
        ]]
    })
 
    // replace all origins with scaled
    // TODO: make this a function
    var origins = importer
        .regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
        .map(o => o.trim().split(/\s+/ig)
            .map(n => (n.includes('.')
                ? parseFloat(n.trim())
                : parseInt(n.trim()))))
    
    origins = origins.concat.apply(origins, brushes)
        .filter(o => o && isFinite(o[0]))
    
    return [[
        Math.min.apply(null, origins.map(b => b[0])),
        Math.min.apply(null, origins.map(b => b[1])),
        Math.min.apply(null, origins.map(b => b[2]))
    ], [
        Math.max.apply(null, origins.map(b => b[0])),
        Math.max.apply(null, origins.map(b => b[1])),
        Math.max.apply(null, origins.map(b => b[2]))
    ]]
}


function addSkybox(fileName) {
    var file
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
    } else {
        file = fileName
    }

    var brushes = importer.regexToArray(/\{[^\{}]*?\}\s*/ig, file)
    brushes.forEach(b => {
        if(b.includes('/sky')) {
            file = file.replace(b, '')
            return false
        }
        return true
    })
    
    var vs = getBounds(file)
    
    // TODO: use a fancy for loop instead contains each corner and extends towards the next two points?
    var points = [
        [vs[0][0], vs[0][1], vs[0][2]-16],
        [vs[1][0], vs[1][1], vs[0][2]],
        
        [vs[0][0]-16, vs[0][1], vs[0][2]],
        [vs[0][0],    vs[1][1], vs[1][2]],
        
        [vs[0][0], vs[0][1]-16, vs[0][2]],
        [vs[1][0], vs[0][1],    vs[1][2]],
        
        
        [vs[0][0], vs[0][1], vs[1][2]],
        [vs[1][0], vs[1][1], vs[1][2]+16],
        
        [vs[1][0],    vs[0][1], vs[0][2]],
        [vs[1][0]+16, vs[1][1], vs[1][2]],
        
        [vs[0][0], vs[1][1],    vs[0][2]],
        [vs[1][0], vs[1][1]+16, vs[1][2]],
        
    ]
    var newBrush = ``
    for(var i = 0; i < points.length / 2; i++) {
        var p1 = points[i*2]
        var p2 = points[i*2+1]
        newBrush += `
{ // brush 0
( ${p1[0]} ${p1[1]} ${p2[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) ( ${p1[0]} ${p2[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p2[0]} ${p2[1]} ${p2[2]} ) ( ${p2[0]} ${p2[1]} ${p1[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p2[0]} ${p1[1]} ${p2[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p1[0]} ${p2[1]} ${p2[2]} ) ( ${p1[0]} ${p2[1]} ${p1[2]} ) ( ${p2[0]} ${p2[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p1[0]} ${p2[1]} ${p1[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p1[0]} ${p1[1]} ${p2[2]} ) ( ${p1[0]} ${p2[1]} ${p2[2]} ) ( ${p2[0]} ${p2[1]} ${p2[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
}
`
    }
    

    var exp = (/\{*\s*\/\/\s*brush\s*0\s*\{*/ig)
    var match = exp.exec(file)
    var pos = exp.lastIndex
    file = file.substr(0, pos - match[0].length) + newBrush + file.substr(pos - match[0].length)
    
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        console.log(`writing ${fileName}`)
        fs.writeFileSync(fileName, file)
    } else {
        return file
    }
}

module.exports = {
    addSkybox,
    getBounds
}



#### draw hints from x and y coordinates?

draw hints in map?



In [ ]:
var path = require('path')
var importer = require('../Core')
var {getBounds} = importer.import('add skybox to map')


var MAPS_HINTS = {
    'base1-a': [
        [-648, -209],
        [-648, 235],
        [-400, 235],
        [-400, 514],
        [300, 514],
        [300, 1922],
        [9999, 1922],
        [9999, -9999],
        [-648, -9999],
        [-648, -209]
    ],
    'base1-b': [
        [-648, -209],
        [-648, 235],
        [-400, 235],
        [-400, 514],
        [300, 514],
        [300, 1922],
        [-1300, 2174],
        [-1300, 648],
        [-648, -209],
    ],
    'base1-c': [
        [-1300, 1000],
        [-1300, 2174],
        [-9999, 9999],
        [-9999, 1000],
        [-1300, 1000],
    ],
    
    'base2-a': [
        [-1212, 1242],
        [340, 1242],
        [340, 1030],
        [1324, 1030],
        [1324, 3109],
        [-1212, 3109],
        [-1212, 1242]
    ],
    'base2-b': [
        [-1212, 1242],
        [340, 1242],
        [340, 1030],
        [1324, 1030],
        [1324, -1711],
        [736, -1711],
        [736, -1251],
        [-1212, -1251],
        [-1212, 1242]
    ],
    'base2-c': [
        [1324, 1030],
        [1324, -1711],
        [736, -1711],
        [736, -1251],
        [-1212, -1251],
        [-1212, -2815],
        [1324, -2815],
        [1324, 1030],
    ]
}

function addHints(fileName, hints) {
    var file
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
        if(!hints) {
            hints = path.basename(fileName).replace(/[-_]converted|\.map$/ig, '')
        }
    } else {
        file = fileName
    }
    
    if(!Array.isArray(hints)) {
        var exp = new RegExp(hints, 'ig')
        hints = Object.keys(MAPS_HINTS).reduce((arr, k) => {
            if(k.match(exp)) {
                arr.push(MAPS_HINTS[k])
            }
            return arr
        }, [])
    }

    var brushes = importer.regexToArray(/\{[^\{}]*?\}\s*/ig, file)
    
    /*
    brushes.forEach(b => {
        if(b.includes('/areaportal')) {
            file = file.replace(b, '')
            return false
        }
        return true
    })
    */
    
    var vs = getBounds(file)
    
    var newBrush = ``
    
    for(var h = 0; h < hints.length; h++) {
        for(var i = 0; i < hints[h].length - 1; i++) {
            var minX = Math.min(hints[h][i][0], hints[h][i+1][0]), 
                maxX = Math.max(hints[h][i][0], hints[h][i+1][0]), 
                minY = Math.min(hints[h][i][1], hints[h][i+1][1]), 
                maxY = Math.max(hints[h][i][1], hints[h][i+1][1])
            var p1 = [minX,   minY, vs[0][2]]
            var p2 = [maxX+2, maxY+2, vs[1][2]]

            newBrush += `
{ // brush 0
( ${p1[0]} ${p1[1]} ${p2[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) ( ${p1[0]} ${p2[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p2[0]} ${p2[1]} ${p2[2]} ) ( ${p2[0]} ${p2[1]} ${p1[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p2[0]} ${p1[1]} ${p2[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p1[0]} ${p2[1]} ${p2[2]} ) ( ${p1[0]} ${p2[1]} ${p1[2]} ) ( ${p2[0]} ${p2[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p1[0]} ${p2[1]} ${p1[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p1[0]} ${p1[1]} ${p2[2]} ) ( ${p1[0]} ${p2[1]} ${p2[2]} ) ( ${p2[0]} ${p2[1]} ${p2[2]} ) common/hint 0 0 0 1 1 0 0 0
}
`
        }
    }
    
    var exp = (/\{*\s*\/\/\s*brush\s*0\s*\{*/ig)
    var match = exp.exec(file)
    var pos = exp.lastIndex
    file = file.substr(0, pos - match[0].length) + newBrush + file.substr(pos - match[0].length)
    
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        console.log(`writing ${fileName}`)
        fs.writeFileSync(fileName, file)
    } else {
        return file
    }
}

module.exports = addHints
module.exports.addHints = addHints
module.exports.MAPS_HINTS = MAPS_HINTS


#### split map using hints and portals?

copy brushes inside a polygon?

split map hints?


In [ ]:
var path = require('path')
var importer = require('../Core')
var {MAPS_HINTS, addHints} = importer.import('draw hints in map')
var translateMap = importer.import('translate quake map')
var {isInside} = importer.import('brush to vertex')
var {addSkybox} = importer.import('add skybox to map')

var BUFFER_UNITS = 200
var ENTITIES_EXP = /\{[^\{\}]*?"classname"\s*"([^"]*)"[^\{\}]*?(\{[^\{\}]*?\}[^\{\}]*?)*\}/ig
var BRUSH_EXP = /\{[^\{\}]*?\}\s*/ig
var moves = [
    [0, 0],
    [BUFFER_UNITS, 0],
    [0, BUFFER_UNITS],
    [BUFFER_UNITS, BUFFER_UNITS],
    [-BUFFER_UNITS, 0],
    [0, -BUFFER_UNITS],
    [-BUFFER_UNITS, -BUFFER_UNITS],
]
function splitHints(fileName, hints) {
    var file
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
        if(!hints) {
            hints = path.basename(fileName).replace(/[-_]converted|\.map$/ig, '')
        }
    } else {
        file = fileName
    }
    
    if(typeof hints !== 'object') {
        var exp = new RegExp(hints, 'ig')
        hints = Object.keys(MAPS_HINTS).reduce((obj, k) => {
            if(k.match(exp)) {
                obj[k] = MAPS_HINTS[k]
            }
            return obj
        }, {})
    }
    
    var multimaps = {}
    var worldspawn = {}
    Object.keys(hints).forEach(k => (multimaps[k] = [], worldspawn[k] = []))
    
    var entities = importer
        .regexToArray(ENTITIES_EXP, file, false)
    
    entities.forEach(ent => {
        var include = false
        var points = []
        ent.points = points
        
        var brushes = importer.regexToArray(BRUSH_EXP, ent[0])

        brushes.forEach(brush => {
            var pts = importer
                .regexToArray(/\(((\s*[0-9\.-]+\s*)*)\)/ig, brush, 1)
                .map(m => m.trim().split(/\s+/ig)
                    .map(n => (n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim()))))
            
            if(brush.includes('/sky')) {
                return
            }

            pts.forEach(b => {
                Object.keys(hints).forEach(k => {
                    if(worldspawn[k].indexOf(brush) === -1) {
                        // if moving the point in any direction would
                        //   cause it to be inside, include that brush
                        for(var m = 0; m < moves.length; m++) {
                            if(isInside(hints[k].map(h => ({x: h[0], y: h[1]})), 
                                        hints[k].length-1, 
                                        {x: b[0] + moves[m][0], y: b[1] + moves[m][1]})) {
                                points.push.apply(pts)
                                worldspawn[k].push(brush) // object reference points added below
                                return
                            }
                        }
                    }
                })
            })
        })

        var skipOrigin = false
        Object.keys(worldspawn).forEach(k => {
            if(!worldspawn[k].length) return

            multimaps[k].push({
                '0': ent[0].replace(BRUSH_EXP, '').replace('}', worldspawn[k].join('\n') + '}'),
                points: points
            })
            worldspawn[k] = []
            skipOrigin = true
        })
        if(skipOrigin) return // already know its a brush type not point type

        var origins = importer
            .regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, ent[0], 1)
            .map(o => o.trim().split(/\s+/ig)
                .map(n => (n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim()))))
        
        origins.forEach(b => {
            Object.keys(hints).forEach(k => {
                // if moving the point in any direction would
                //   cause it to be inside, include that brush
                for(var m = 0; m < moves.length; m++) {
                    if(isInside(hints[k].map(h => ({x: h[0], y: h[1]})), 
                                hints[k].length-1, 
                                {x: b[0] + moves[m][0], y: b[1] + moves[m][1]})) {
                        points.push(b)
                        multimaps[k].push(ent)
                    }
                }
            })
        })
    })
    
    // get middle 50% of all points
    var translations = Object.keys(multimaps).map(k => {
        var result = []
        for(var i = 0; i < 3; i++) {
            var points = [].concat.apply([], multimaps[k]
                .map(ent => ent.points.map(p => p[i])))
            points.sort()
            result[i] = -Math.round((points
                // get the middle half
                .slice(points.length / 4, points.length / 4 * 3)
                // get the average
                .reduce((a, b) => a + b, 0) / (points.length / 2))
                // round to the nearest grid position?
                / 147.2) * 147.2
        }
        return result
    })
    
    console.log(translations)
    
    // copy all the brushes in to a new file, skybox and save
    var maps = Object.keys(multimaps).forEach((k, i) => {
        var map = multimaps[k]
            .filter((ent, i, arr) => arr.indexOf(ent) === i)
            .map(ent => ent[0])
            .join('\n')
        
        map = translateMap(map, translations[i])
        
        map = addSkybox(map)
        
        file = addHints(file, [hints[k]])
        
        if(typeof fileName === 'string' && fs.existsSync(fileName)) {
            var newFile = path.join(path.dirname(path.resolve(fileName)), path.basename(k) + '.map')
            console.log(`writing ${newFile}`)
            fs.writeFileSync(newFile, map)
        } else {
            return map
        }
    })
    
    return maps
}

module.exports = splitHints


#### TODO: convert quake 2 entities to quake 3?


TODO: use "bodyque" for dead body model explosions? these have a high health in quake 2, they should drop the goods.





```
Quake 1:
------------------------------------------------
func_plat                      *
func_door                      *
info_null                      *
info_player_deathmatch         *
item_health                    *
info_teleport_destination      target_position
item_armor1                    item_armor_combat
item_armor2                    item_armor_combat
item_armorInv                  item_armor_body
item_artifact_envirosuit       item_enviro
item_artifact_invisibility     item_invis
item_artifact_super_damage     item_quad
item_cells                     ammo_lightning
item_rockets                   ammo_rockets
item_shells                    ammo_shells
item_spikes                    ammo_cells
weapon_nailgun                 weapon_plasmagun
weapon_supernailgun            weapon_plasmagun
weapon_supershotgun            weapon_shotgun
weapon_rocketlauncher          *
light                          *
trigger_hurt                   *
trigger_multiple               *
trigger_push                   *
trigger_teleport               *
weapon_grenadelauncher         *
weapon_lightning               *

Quake 2:
-------------------------------------------------
ammo_bullets               *
ammo_cells                 *
ammo_grenades              *
ammo_rockets               *
ammo_shells                *
ammo_slugs                 *
func_button                *
func_door                  *
func_group                 *
func_plat                  *
func_rotating              *
func_timer                 *
func_train                 *
info_notnull               *
info_null                  *
info_player_deathmatch     *
info_player_intermission   *
item_armor_body            *
item_armor_combat          *
item_armor_shard           *
item_enviro                *
item_health                *
item_health_large          *
item_health_mega           *
item_health_small          *
item_quad                  *
light                      *
misc_teleporter_dest       *
path_corner                *
target_laser               *
target_speaker             *
trigger_always             *
trigger_hurt               *
trigger_multiple           *
trigger_push               *
weapon_bfg                 *
weapon_chaingun            weapon_lightning
misc_teleporter            trigger_teleport 
weapon_hyperblasters       weapon_plasmagun
weapon_supershotgun        weapon_shotgun
weapon_grenadelauncher     *
weapon_machinegun          *
weapon_railgun             *
weapon_rocketlauncher      *
weapon_shotgun             *
```


### misc map/bsp scripts



#### get vertex from brush strokes?

brush to vertex?


In [ ]:
  
// Given three colinear points p, q, r, the function checks if 
// point q lies on line segment 'pr' 
function onSegment(p, q, r) 
{
    if (q.x <= Math.max(p.x, r.x) && q.x >= Math.min(p.x, r.x) && 
            q.y <= Math.max(p.y, r.y) && q.y >= Math.min(p.y, r.y)) {
        return true; 
    }
    return false; 
} 
  
// To find orientation of ordered triplet (p, q, r). 
// The function returns following values 
// 0 --> p, q and r are colinear 
// 1 --> Clockwise 
// 2 --> Counterclockwise 
function orientation(p, q, r) 
{ 
    var val = (q.y - p.y) * (r.x - q.x) - 
              (q.x - p.x) * (r.y - q.y); 
  
    if (val == 0) return 0;  // colinear 
    return (val > 0)? 1: 2; // clock or counterclock wise 
} 
  
// The function that returns true if line segment 'p1q1' 
// and 'p2q2' intersect. 
function doIntersect(p1, q1, p2, q2) 
{ 
    // Find the four orientations needed for general and 
    // special cases 
    var o1 = orientation(p1, q1, p2); 
    var o2 = orientation(p1, q1, q2); 
    var o3 = orientation(p2, q2, p1); 
    var o4 = orientation(p2, q2, q1); 
  
    // General case 
    if (o1 != o2 && o3 != o4) {
        return true
    }
  
    // Special Cases 
    // p1, q1 and p2 are colinear and p2 lies on segment p1q1 
    if (o1 == 0 && onSegment(p1, p2, q1)) return true
  
    // p1, q1 and p2 are colinear and q2 lies on segment p1q1 
    if (o2 == 0 && onSegment(p1, q2, q1)) return true
  
    // p2, q2 and p1 are colinear and p1 lies on segment p2q2 
    if (o3 == 0 && onSegment(p2, p1, q2)) return true
  
     // p2, q2 and q1 are colinear and q1 lies on segment p2q2 
    if (o4 == 0 && onSegment(p2, q1, q2)) return true
  
    return false; // Doesn't fall in any of the above cases 
} 


// Returns true if the point p lies inside the polygon[] with n vertices 
function isInside(polygon, n, p) 
{ 
    // There must be at least 3 vertices in polygon[] 
    if (n < 3)  return false; 
  
    // Create a point for line segment from p to infinite 
    var extreme = {x: Number.MAX_VALUE, y: p.y}; 
  
    // Count intersections of the above line with sides of polygon 
    var count = 0, i = 0; 
    do
    { 
        var next = (i+1)%n; 
  
        // Check if the line segment from 'p' to 'extreme' intersects 
        // with the line segment from 'polygon[i]' to 'polygon[next]' 
        if (doIntersect(polygon[i], polygon[next], p, extreme)) 
        { 
            // If the point 'p' is colinear with line segment 'i-next', 
            // then check if it lies on segment. If it lies, return true, 
            // otherwise false 
            if (orientation(polygon[i], p, polygon[next]) == 0) 
               return onSegment(polygon[i], p, polygon[next]); 
  
            count++; 
        } 
        i = next; 
    } while (i != 0); 
  
    // Return true if count is odd, false otherwise 
    return (count%2 == 1);
} 


module.exports = {
    doIntersect,
    isInside
}


#### TODO: 



#### list noises in a quake 3 map?



In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')

function listNoise() {
    var contents = fs.readFileSync('/Users/briancullinan/.q3a/baseq3/baseq2.pk3dir/maps/base1.map').toString('utf8')
    var noises = importer.regexToArray(/noise.*?"\s*"(.*?)"/ig, contents, 1)
    return {all: noises, missing: noises.filter(n => {
        return !fs.existsSync(path.join('/Users/briancullinan/.q3a/baseq3/baseq2.pk3dir/sound/', n))
    })}
}

module.exports = listNoise